### Prevendo valores de ações com algoritmo de LSTM

O dataset é composto pelo histórico do valor das ações da Amazon no período de 04/01/2010 até 04/01/2019. 

Link para acessar dados do Yahoo Finanças:

https://br.financas.yahoo.com/quote/MSFT/history?period1=1420070400&period2=1652918400&interval=1d&filter=history&frequency=1d&includeAdjustedClose=true


**Objetivo:** Prever valor de ações com base em dados históricos.

### Importando as bibliotecas necessárias:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import LSTM,Dense

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from keras.models import load_model
from keras.preprocessing.sequence import TimeseriesGenerator

import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
import plotly.graph_objects as go
%matplotlib inline

### Subindo o dataframe:

In [ ]:
df = pd.read_csv('AMZN.csv')
df['Date'] = pd.to_datetime(df['Date']) #realizando a conversão da data para formato datetime

In [ ]:
df.head()

In [ ]:
df.shape #analisando linhas e colunas

In [ ]:
df.tail() #analisando últimos dados da nossa base

### Visualizando os valores de fechamento da Amazon

In [ ]:
data = df['Date'].to_numpy()
close = df['Close'].to_numpy()

plt.figure(figsize = (15,10))
plt.plot(data, close, label='Amazon')

plt.legend(loc='best')
plt.show()

### Média Móvel

Um algoritmo clássico para este tipo de problema é o da Média Móvel. Ele consiste em utilizar a **média de N dias observados para prever o próximo dia**. Vamos utilizar esta técnica para um N de 10 e de 20 dias.


In [ ]:
#Filtrando os dados
df = pd.DataFrame(df[['Close','Date']])
df.set_index('Date', inplace=True)

In [ ]:
df.head()

In [ ]:
df['MA_window_10'] = df['Close'].rolling(10).mean().shift() #média móvel em 10 dias
df['MA_window_20'] = df['Close'].rolling(20).mean().shift() #média móvel em 20 dias

#Rolling para deslocamento.Este parâmetro especifica o número de observações usadas para calcular a estatística
#shift é utilizado para deslocar o índice de DataFrame por um número especificado de períodos com uma freqüência de tempo opcional

### Visualizando o resultado:

In [ ]:
plt.figure(figsize=(15,10))
plt.grid(True)
plt.plot(df['Close'], label='Close')
plt.plot(df['MA_window_10'], label='MA window 10 days')
plt.plot(df['MA_window_20'], label='MA window 20 days')
plt.legend(loc=2)
plt.show()

### Visualizando os dados mais recentes: últimos 300 dias

In [ ]:
limit = 300

plt.figure(figsize=(15,10))
plt.grid(True)
plt.plot(df['Close'][-limit:], label='Close')
plt.plot(df['MA_window_10'][-limit:], label='MA window 10 days')
plt.plot(df['MA_window_20'][-limit:], label='MA window 20 days')
plt.legend(loc=2)
plt.show()

### Prevendo os "N" dias com base na média móvel aritmética

In [ ]:
df['MA_window_10_forward_10'] = np.NaN #preechendo com NaN os valores da coluna de MA_window_10_forward_10

In [ ]:
def make_window(window_size, start_point):
    return [start_point+x for x in range(window_size)]  #realizando a janela de tempo

In [ ]:
window_size = 10
forward_days = 10 
for index in range(window_size, len(df), forward_days):
    for i in range(0,forward_days):
        if index+i >= len(df):
            break
        window_close = make_window(window_size-i, index+i-window_size)
        #print(window_close)
        window_MA = make_window(i, index)
        #print(window_MA)
        mean = pd.concat([ df['Close'].iloc[window_close], df['MA_window_10_forward_10'].iloc[window_MA] ]).mean(axis=0)
        df.iat[index+i, df.columns.get_loc('MA_window_10_forward_10')] = mean

Plotando os dados:

In [ ]:
plt.figure(figsize = (15,10))

size = len(df)-limit - (len(df)-limit)%forward_days

for index in range(size, len(df), forward_days):
    plt.plot(df['MA_window_10_forward_10'][index:index+forward_days], color='r')
    
plt.plot(df['Close'][-limit:], color='b', label='Close')
#plt.legend(loc='best')
plt.show()

Note que cada reta vermelha no gráfico representa uma previsão de 10 dias, baseado nos 10 dias anteriores. Por isso elas são descontínuas.

Este tipo de abordagem é muito simplista. Isto porque o verdadeiro objetivo é prever N dias a frente para ver qual será o comportamento da ação. E ambos algoritmos falham nessa função.

### Aplicando o algoritmo LSTM para prever

Vamos agora utilizar uma Rede LSTM para prever o comportamento da empresa.

In [ ]:
df = pd.read_csv('AMZN.csv')
df['Date'] = pd.to_datetime(df['Date']) #realizando a conversão da data para formato datetime
df.set_index(df['Date'], inplace=True)
df.drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)

df.head()

In [ ]:
close_data = df['Close'].values
close_data = close_data.reshape(-1,1) #transformar em array
close_data

### Normalizando os dados

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler = scaler.fit(close_data)
close_data = scaler.transform(close_data)

close_data

### Separação da base de treino e teste:

In [ ]:
#close_data = df['Close'].values
#close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df['Date'][:split]
date_test = df['Date'][split:]

print(len(close_train))
print(len(close_test))

In [ ]:
look_back = 10

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=20)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)

In [ ]:
np.random.seed(7)

model = Sequential()
model.add(LSTM(80, activation='relu', input_shape=(look_back,1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit(train_generator, epochs=num_epochs, verbose=1)

In [ ]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)
layout = go.Layout(
    title = "Predições da Amazon",
    xaxis = {'title' : "Data"},
    yaxis = {'title' : "Fechamento"}
)
fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()

Aplicando a predição nos próximos 15 dias

In [ ]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = df['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 15 #definição dos próximos dias
forecast = predict(num_prediction, model) #resultado de novos dias
forecast_dates = predict_dates(num_prediction)

In [ ]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)
layout = go.Layout(
    title = "Forecast Amazon",
    xaxis = {'title' : "Data"},
    yaxis = {'title' : "Fechamento"}
)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [ ]:
df = pd.DataFrame(df)
df_past = df[['Close']].reset_index()                                      #resetando a data como indice
df_past.rename(columns={'index': 'Date', 'Close': 'Actual'}, inplace=True) #criando nome das colunas
df_past['Date'] = pd.to_datetime(df_past['Date'])                          #configurando para datatime
df_past['Forecast'] = np.nan                                               #Preenchendo com NAs
df_past['Forecast'].iloc[-1] = df_past['Actual'].iloc[-1]   
df_past.head(3)

In [ ]:
df_past.tail()

In [ ]:
# Faz a transformação inversa das predições
forecast = forecast.reshape(-1, 1) #reshape para array
forecast = scaler.inverse_transform(forecast)

In [ ]:
df_future = pd.DataFrame(columns=['Date', 'Actual', 'Forecast'])
df_future['Date'] = forecast_dates
df_future['Forecast'] = forecast.flatten()
df_future['Actual'] = np.nan
df_future.head()

In [ ]:
df_future.tail()

In [ ]:
#results = df_past.append(df_future).set_index('Date')
results = pd.concat([df_past, df_future]).set_index('Date')
results.head()
#results.plot(title='Amazon Forecast')

In [ ]:
results.tail()

In [ ]:
results.plot(title='Amazon Forecast')

In [ ]:
results2018 =  results.loc['2018-01-01':]

In [ ]:
plot_data = [
    go.Scatter(
        x=results2018.index,
        y=results2018['Actual'],
        name='actual'
    ),
    go.Scatter(
        x=results2018.index,
        y=results2018['Forecast'],
        name='prediction'
    )
]

plot_layout = go.Layout(
        title='Forecast Amazon'
    )
fig = go.Figure(data=plot_data, layout=plot_layout)

fig.show()

import plotly as ply
ply.offline.plot(fig)